### Task 0 Before your go

> 1. Rename Assignment-02-###.ipynb where ### is your student ID.
> 2. The deadline of Assignment-02 is 23:59pm, 04-21-2024
> 3. In this assignment, you will use word embeddings to explore our Wikipedia dataset.

### Task 1 Train word embeddings using SGNS 
> Use our enwiki-train.json as training data. You can use the [Gensim tool](https://radimrehurek.com/gensim/models/word2vec.html). But it is recommended to implement by yourself. You should explain how hyper-parameters such as dimensionality of embeddings, window size, the parameter of negative sampling strategy, and initial learning rate have been chosen.

In [36]:
# import some necessary libraries
from typing import List
from collections import defaultdict
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim import utils
import json
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import heapq

In [3]:
# load the train and test data from the json file

# NOTE: The function is inherited from my solution of assignment 1
def load_json(file_path: str) -> list:
    """
    Fetch the data from `.json` file and concat them into a list.

    Input:
    - file_path: The relative file path of the `.json` file

    Returns:
    - join_data_list: A list containing the data, with the format of [{'title':<>, 'label':<>, 'text':<>}, {}, ...]
    """
    join_data_list = []
    with open(file_path, "r") as json_file:
        for line in json_file:
            line = line.strip()
            # guaranteen the line is not empty
            if line: 
                join_data_list.append(json.loads(line))
    return join_data_list

train_file_path, test_file_path = "enwiki-train.json", "enwiki-test.json"
train_data_list, test_data_list = map(load_json, [train_file_path, test_file_path])

class Corpus:
    def __iter__(self):
        for line in train_data_list:
            yield utils.simple_preprocess(line["text"])


In [4]:
class MyWord2Vec:
    def __init__(self, text: List[dict], dimensionality: int=100, window_size: int=5, negative_samples: int=5, lr: float=0.001) -> None:
        """
        Args:
        - text: The training data
        - dimensionality: The dimension of the word embeddings
        - window_size: The size of the context window
        - negative_samples: The number of negative samples
        - lr: Learning rate of the algorithm
        """
        self.dim = dimensionality
        self.window = window_size
        self.neg = negative_samples
        self.lr = lr
        self.__vocab = set()
        self.__word_frq = defaultdict(int)
        self.__word2idx = {}
        self.__idx2word = {}
        self.__embedding = None
        self.__context_words = []
        self.__context_targets = []
        self.__build(text)
        

    def __preprocess(self, text: List[str]) -> None:
        """
        Calculate the vocabulary and the frequency of each word in the training data, while maintaining the (idx, word) map.
        """
        for sample in text:
            words = sample["text"].split()
            for word in words:
                self.__vocab.add(word)
                self.__word_frq[word] += 1
        for idx, word in enumerate(self.__vocab):
            self.__word2idx[word] = idx
            self.__idx2word[idx] = word

    def __generate_training_data(self, text: List[str]) -> None:
        """
        Generate training data from each window and save them in `self.__context_words` and `self.__context_targets`
        """
        for sample in text:
            words = sample["text"].split()
            for i, curr_word in enumerate(words):
                # the "window" around the current world
                for j in range(max(0, i - self.window), min(i + self.window + 1, len(words))):
                    if i != j:
                        self.__context_words.append(self.__word2idx[curr_word])
                        self.__context_targets.append(self.__word2idx[words[j]])

    def __initialize_embedding(self) -> None:
        """
        Initialize the embedding matrix with random values
        """
        self.__embedding = np.random.uniform(-0.5 / self.dim, 0.5 / self.dim, size=(len(self.__vocab), self.dim))
    
    def __build(self, text: List[map]) -> None:
        """
        Compute and store the relevant information of the training data in the class
        """
        self.__preprocess(text)
        self.__generate_training_data(text)
        self.__initialize_embedding()

    def train(self, epochs: int=5) -> None: 
        for epoch in range(epochs):
            # learning rate decay
            learning_rate = self.lr * (1 - epoch / epochs)

            print("Training Epoch: %d" % (epoch + 1))

            for context_word, target_word in zip(self.__context_words, self.__context_targets):
                context_vector = self.__embedding[context_word]
                target_vector = self.__embedding[target_word]

                # positive sample update
                score = np.dot(target_vector, context_vector)
                exp_score = math.exp(score)
                grad_context = (exp_score / (1 + exp_score) - 1) * target_vector
                grad_target = (exp_score / (1 + exp_score) - 1) * context_vector
                self.__embedding[context_word] -= learning_rate * grad_context
                self.__embedding[target_word] -= learning_rate * grad_target

                # negative sample update
                for _ in range(self.neg):
                    negative_word = random.randint(0, len(self.__vocab) - 1)
                    if negative_word != target_word:
                        negative_vector = self.__embedding[negative_word]
                        score = np.dot(negative_vector, context_vector)
                        exp_score = math.exp(score)
                        grad_context = exp_score / (1 + exp_score) * negative_vector
                        grad_target = exp_score / (1 + exp_score) * context_vector
                        self.__embedding[context_word] -= learning_rate * grad_context
                        self.__embedding[target_word] -= learning_rate * grad_target


In [5]:
sentence = Corpus()
model = Word2Vec(
    sentences=sentence, vector_size=100, alpha=0.025, window=5, min_count=5, sample=0.001, 
    seed=1, workers=3, min_alpha=0.0001, sg=1, negative=5, ns_exponent=0.75, epochs=5, 
    sorted_vocab=1
)

### Task 2 Find similar/dissimilar word pairs

> Randomly generate 100, 1000, and 10000-word pairs from the vocabularies. For each set, print 5 closest word pairs and 5 furthest word pairs (you can use cosine-similarity to measure two words). Explain your results.

In [6]:
def generate_random_paris(samples: int):
    """
    Generate random indices without replacement, then pairs the indices to get word pairs
    """
    indices = random.sample(range(len(model.wv)), 2 * samples)
    indices1, indices2 = indices[:samples], indices[samples:]
    return [model.wv.index_to_key[i] for i in indices1], [model.wv.index_to_key[i] for i in indices2]

def find_closest_furthest(num: int=5, words1: List[str]=None, words2: List[str]=None) -> None:
    """
    Find the cloest/furthest word pairs using `model.wv.similarity`.

    Here a heap queue is used to reduce time complexity to $O(n\log k)$, where k denotes the `num`
    """
    heap = []
    for i in range(len(words1)):
        # compute the similarity and push it into the heap
        heapq.heappush(heap, (model.wv.similarity(words1[i], words2[i]), words1[i], words2[i]))
    return heapq.nlargest(num, heap), heapq.nsmallest(num, heap)[::-1]

def print_word_pairs(results: List[tuple], flag: str) -> None:
    """
    Print the result in formatted string
    """
    print("The 5 {:>8} word pairs:".format(flag))
    for result in results:
       print("Word pairs: ({:>15}, {:>15}) --> Similarity: {:>8.6f}".format(result[1], result[2], result[0]))


random.seed(408)
pairs = [100, 1000, 10000]
for pair in pairs:
    print("For {:>5} random pairs from the vocabularies:".format(pair))
    cloest, furthest = find_closest_furthest(5, *generate_random_paris(pair))
    print_word_pairs(cloest, "closest")
    print_word_pairs(furthest, "furthest")
    print("-" * 70)

For   100 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (      herodotus,       sorcerers) --> Similarity: 0.880446
Word pairs: (stereotypically,             een) --> Similarity: 0.842470
Word pairs: (        bruxing,        plumbers) --> Similarity: 0.839791
Word pairs: (           sown,     prohibitive) --> Similarity: 0.833044
Word pairs: (         shaded,           booby) --> Similarity: 0.825319
The 5 furthest word pairs:
Word pairs: (         member,        investor) --> Similarity: 0.233117
Word pairs: (          harry,           kuala) --> Similarity: 0.231424
Word pairs: (        trapped,          arabia) --> Similarity: 0.216789
Word pairs: (       superman,         outputs) --> Similarity: 0.189746
Word pairs: (     separately,       communism) --> Similarity: 0.149030
----------------------------------------------------------------------
For  1000 random pairs from the vocabularies:
The 5  closest word pairs:
Word pairs: (        itching,       

### Task 3 Present a document as an embedding

> For each document, you have several choices to generate document embedding: 1. Use the average of embeddings of all words in each document; 2. Use the first paragraph’s words and take an average on these embeddings; 3. Use the doc2vec algorithm to present each document. Do the above for both training and testing dataset

In [19]:
####################################################################################
###        1. Use the average of embeddings of all words in each document        ###
####################################################################################
def print_document_embeddings(embeddings: dict, display: int=10) -> None:
    """
    Print the first `display` embeddings, default value is 10
    """
    count = 0
    for title, embedding in embeddings.items():
        if count >= display:
            break
        count += 1
        print("{}:\n{}".format(title, embedding))
        print("-" * 70)

def average_all_words() -> dict:
    doc_embeddings = {}
    for line in train_data_list:
        line_text = utils.simple_preprocess(line["text"]) # preprocess the text as in class `Corpus`
        # doc title and doc embedding computed by averaging all words embeddings
        doc_title = line["title"]
        valid_word_embeddings = [model.wv[word] for word in line_text if word in model.wv]
        doc_embedding = sum(valid_word_embeddings) / len(valid_word_embeddings)
        # store the information in a dictionary
        doc_embeddings[doc_title] = doc_embedding

    return doc_embeddings

average_all_words_embedding = average_all_words()
print_document_embeddings(average_all_words_embedding)


Citizen_Kane:
[-0.18555687 -0.11498394  0.12703045  0.19868466 -0.08605349 -0.3689179
  0.21345545  0.31779507 -0.16548374 -0.25047037 -0.00524598 -0.33721638
  0.00063661  0.08648021  0.01215639 -0.0763451   0.30754155  0.08644778
 -0.04686667 -0.37171257 -0.14886318  0.14374806  0.09710665 -0.04084833
  0.03245828  0.13903828 -0.27641234  0.1391886  -0.13908271 -0.18599151
 -0.17242636  0.12265657 -0.09491661  0.02495716 -0.07464156  0.11531783
 -0.12658444 -0.2402224  -0.12558994 -0.13856143 -0.0395303  -0.08800881
 -0.30476314 -0.13202067  0.17026316  0.02671698 -0.21354283  0.05506658
  0.19191238  0.02331443  0.11432122 -0.01052564 -0.28179178 -0.06251904
 -0.07604388 -0.03214294 -0.13627957 -0.02698852 -0.00246957  0.13575526
  0.15388337  0.11309912  0.18228544  0.01188574 -0.00392627  0.07238039
 -0.03351165  0.2826619  -0.25783497  0.19132069  0.21434167  0.260049
  0.02967949 -0.01430467  0.15401529  0.22354499 -0.05086841  0.13207738
  0.19135456 -0.03810756 -0.18478478 -0.

In [34]:
####################################################################################
###  2. Use the first paragraph’s words and take an average on these embeddings  ###
####################################################################################
def find_first_paragraph(text: str):
    return text.split("\n")[0]

def average_first_para_words() -> dict:
    doc_embeddings = {}
    for line in train_data_list:
        doc_title = line["title"]
        valid_word_embeddings = [model.wv[word] for word in utils.simple_preprocess(find_first_paragraph(line["text"])) if word in model.wv]
        # since the number of words in the first paragraph is so small, it may occur `ZeroDivisonError` in the computation, here I use a 
        # try-except flow to handle this exception
        try:
            doc_embedding = sum(valid_word_embeddings) / len(valid_word_embeddings)
        except:
            doc_embedding = [0 for _ in range(len(valid_word_embeddings))]
        doc_embeddings[doc_title] = doc_embedding

    return doc_embeddings

average_first_para_words_embedding = average_first_para_words()
print_document_embeddings(average_first_para_words_embedding)

Citizen_Kane:
[-0.23090884 -0.13291559  0.20529905  0.21812418 -0.0190186  -0.4826053
  0.24806686  0.25670743 -0.22496761 -0.26894984 -0.00441595 -0.3581196
  0.02116748  0.00248322  0.0051759  -0.14579874  0.2945101  -0.01452728
 -0.00802334 -0.3153541  -0.20796755  0.19100672  0.05338027 -0.13419762
 -0.01046245  0.10917778 -0.3302613   0.12475839 -0.21306188 -0.17888463
 -0.1695718   0.10462276 -0.1263146   0.06113715 -0.04916075  0.16068289
 -0.22512802 -0.2268536  -0.17277665 -0.1014448  -0.01882456 -0.13939385
 -0.31199345 -0.08741815  0.14783925  0.05809337 -0.19782293  0.07757351
  0.10127524  0.07007059  0.18039902 -0.02828163 -0.2131288  -0.06944453
 -0.14726196 -0.15870269 -0.10337288 -0.01378004  0.04356508  0.10337719
  0.19568619  0.19663046  0.22753777 -0.00818827 -0.03096233  0.06069575
 -0.0204902   0.29396346 -0.26438767  0.21963672  0.20457593  0.23003624
  0.08429366 -0.01530797  0.11847984  0.21866457 -0.05647788  0.19103517
  0.16344292 -0.00218383 -0.11557421 -0

In [41]:
####################################################################################
###             3. Use the doc2vec algorithm to present each document            ###
####################################################################################
def build_doc2vec() -> Doc2Vec:
    model = Doc2Vec(vector_size=100, window=5, min_count=5, epochs=10)
    model.build_vocab(tagged_data)
    model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

def print_doc2vec_embedding(model: Doc2Vec, display: int=10) -> None:
    for index, line in enumerate(train_data_list):
        if index >= display:
            break
        print("{}:\n{}".format(line["title"], model.infer_vector(line["text"].split())))
        print("-" * 70)

tagged_data = [TaggedDocument(words=doc.split(), tags=[str(i)]) for i, doc in enumerate([line["text"] for line in train_data_list])]

doc_model = build_doc2vec()
print_doc2vec_embedding(doc_model)

Citizen_Kane:
[ 4.7364583  -0.4052419  -3.325441   -0.00980055  2.2403915  -0.38089612
 -1.7430489   1.9974115  -0.34079576 -1.8416704   4.6921206  -0.83687544
 -0.5875558  -2.5266826  -1.5581137  -1.9378619   0.41974545 -0.20577174
 -0.233671   -0.5592773  -1.2667687  -2.4095256   2.1755638   1.3998338
 -0.37822956 -2.763966    0.19035669  0.38866246  0.65558684 -1.5939412
  0.19869779 -0.18014884 -0.6731388   2.7298503  -1.7768732  -1.2059423
  0.71505576 -4.7347097  -3.6729484   3.0021446   1.696856   -2.112462
  2.5512295  -4.776504    1.9631578  -2.9360864   2.0496993   0.9342798
  4.219702   -0.23724927  3.6270535   5.600189    2.2999876  -1.7758241
  3.6775427  -1.3034785   1.980849    3.5197618  -2.7026672   2.130448
 -4.533793    3.3864272   3.861147   -3.0859003   1.342131    0.91237205
  2.8728738  -0.33080804 -3.0298274   0.22950499 -3.574138    0.13689718
 -1.1283343  -1.7133118  -1.3141903   1.9607078   4.1194477  -7.1288643
  2.9881096   4.2724433  -3.0735712   3.6197658

### Task 4 Build classifier to test docs
> Build softmax regression model to classifier testing documents based on these training doc embeddings. Does it getting better than Naive Bayes'? (You have 3 models.)

In [ ]:
# Your code




### Task 5 Use t-SNE to project doc vectors

> Use t-SNE to project training document embeddings into 2d and plot them out for each of the above choices. Each point should have a specific color (represent a particular cluster). You may need to try different parameters of t-SNE. One can find more details about t-SNE in this [excellent article](https://distill.pub/2016/misread-tsne/).

In [ ]:
# Your code


